In [3]:
import numpy as np
import pandas as pd
import os
from collections import defaultdict
from random import *

In [4]:
vertex_set = set();
vertex_dict = {}
vertex_followlist ={}
sink_dict = {};
sink_followedlist = defaultdict(list)

with open("train.txt") as trainfile:
    for i, line in enumerate(trainfile):
        line_list = [int(k) for k in line[:-1].split("\t")];
        vertex_dict[line_list[0]]=len(line_list)-1
        vertex_set.add(line_list[0]);
        vertex_followlist[line_list[0]]= line_list[1:]
        for s in line_list[1:]:
            sink_followedlist[s].append(line_list[0])
            if s in sink_dict:
                sink_dict[s] += 1;
            else:
                sink_dict[s] = 1;
        if i % 3000 == 0:
            print(i);

0
3000
6000
9000
12000
15000
18000


In [5]:
new_sink_dict = {};
threshold = 10;
for k in sink_dict:
    if sink_dict[k] >= threshold:
        new_sink_dict[k] = sink_dict[k];
        
new_sink_set = set(new_sink_dict);

print(sink_dict[4867135]);
print(len(vertex_set));
print(len(new_sink_set))

1
20000
394342


In [6]:
test_vertex_and_sink_set = set();

with open("test-public.txt") as testfile:
    for i, line in enumerate(testfile):
        if i == 0:
            continue;
        line_list = [int(k) for k in line[:-1].split("\t")];
        for s in line_list:
            test_vertex_and_sink_set.add(s);
print(len(test_vertex_and_sink_set));

total_set = test_vertex_and_sink_set.union(new_sink_set).union(vertex_set);
# all nodes in new data set
print(len(total_set));

5947
401218


In [7]:
total_dict = {};
total_list = []
for i, p in enumerate(total_set):
    total_dict[p] = i
    total_list.append(p)

In [8]:
max_neighbors = 1000

import numpy as np
total_array = np.array(total_list);

pairs = [];
true_pairs = [];
"""
"""

with open("train.txt") as trainfile:
    for i, line in enumerate(trainfile):
        line_list = [int(k) for k in line[:-1].split("\t")];
        v = line_list[0];
        ranking = [-sink_dict[k] for k in line_list[1:]];
        sorting = np.argsort(ranking);
        filtered_linelist = np.array(line_list[1:])[sorting];
        for s in filtered_linelist[1:max_neighbors]:
            if s in total_set:
                pairs.append([total_dict[v], total_dict[s]]);
        if i % 1000 == 0:
            print(i);

test_pairs = [];
with open("test-public.txt") as testfile:
    for i, line in enumerate(testfile):
        if i == 0:
            continue;
        line_list = [int(k) for k in line[:-1].split("\t")];
        test_pairs.append([line_list[0], total_dict[line_list[1]], total_dict[line_list[2]]]);

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000


In [9]:
correspondence = total_array
for i in pairs:
    true_pairs.append([correspondence[i[0]],correspondence[i[1]]])
true_testpairs = []

for i in test_pairs:
    true_testpairs.append([correspondence[i[1]],correspondence[i[2]]])

print (len(true_pairs))
print (len(true_testpairs))

5168055
2000


In [10]:
np.savez_compressed("filtered_data", correspondence = total_array, pairs = pairs, test_pairs = test_pairs);

In [50]:
temp_new_vertex_followlist = defaultdict(list)
temp_new_sink_followlist = defaultdict(list)
new_vertex_followlist={}
new_sink_followlist={}
new_vertex_followquicklist ={}
new_sink_followquicklist ={}
new_vertex_dict = {}
new_sink_dict = {}
for i in true_pairs:
    temp_new_vertex_followlist[i[0]].append(i[1])
    temp_new_sink_followlist[i[1]].append(i[0])
    new_vertex_followquicklist[i[0],i[1]]=1
    new_sink_followquicklist[i[1],i[0]]=1

for i in temp_new_vertex_followlist:
    new_vertex_followlist[i]=temp_new_vertex_followlist[i]
for i in temp_new_sink_followlist:
    new_sink_followlist[i]=temp_new_sink_followlist[i]    
for i in new_vertex_followlist.keys():
    new_vertex_dict[i]=len(new_vertex_followlist[i])
    for j in new_vertex_followlist[i]:
        if j in new_sink_dict:
            new_sink_dict[j]+=1
        else:   
            new_sink_dict[j]=1
print (len(new_vertex_followlist))
print (len(new_sink_followlist))
print (len(new_vertex_dict))
print (len(new_sink_dict))

19138
319312
19138
319312


In [137]:
def countFollowingFollower(a,b):
    """return 
    1. a following count 
    2. a followed count
    3. b following count
    4. b followed count
    5. a following/a followed
    6. b following/b followed
    """
    count1,count2,count3,count4,count5,count6,count7,count8=0,0,0,0,0,0,0,0
    
    if a in new_vertex_dict:
        count1 = new_vertex_dict[a]
    if b in new_vertex_dict:
        count3 = new_vertex_dict[b]
    if a in new_sink_dict:
        count2 = new_sink_dict[a]
    if b in new_sink_dict:
        count4 = new_sink_dict[b]
    
    if count2!=0:
        count5 = count1/count2
    else:
        count5 = count1
    if count4!=0:
        count6 = count3/count4
    else:
        count6 = count2
#     if b in new_vertex_followlist:    
#         if a in new_vertex_followlist[b]:
#             count7=1
#     if a in vertex_set:    
#         if b in new_vertex_followlist[a]:
#             count8=1
    return (count1,count2,count3,count4,count5,count6)

"""
count8  if a follow b

"""
countFollowingFollower(1406432,2481036)

(63, 6, 13, 23, 10.5, 0.5652173913043478)

In [123]:
# test
def findneighbor(a,b):
    import math

#     c is common friend
#     1.a→c→b
#     2.a→c←b
#     3.a←c→b
#     4.a←c←b
#     5. Jaccard's coeffcient common friends/total friends
#     6. Adamic/Adar 
#     7. commonfriend
#     8. graph distance

    tempA,tempB,tempsinkA,tempsinkB = [],[],[],[]
    count1,count2,count3,count4,count5,count6,count7, count8 = 0,0,0,0,0,0,0,0
# #     a 一定在 vertex_set
    if a in new_vertex_followlist:
        tempA = new_vertex_followlist[a]
    if a in new_sink_followlist:
        tempsinkA =new_sink_followlist[a]
    if b in new_vertex_followlist:
        tempB = new_vertex_followlist[b]
    if b in new_sink_followlist:
        tempsinkB =new_sink_followlist[b]  
    
    if tempA !=None and b in new_sink_followlist:
            count1=len(set(tempA).intersection(new_sink_followlist[b]))
  
    count2 = len(set(tempA).intersection(tempB))
    if a in new_sink_followlist and b in new_sink_followlist:
        count3 = len(set(new_sink_followlist[a]).intersection(new_sink_followlist[b]))
        
    if a in new_sink_followlist and tempB!=None:
        count4 = len(set(new_sink_followlist[a]).intersection(tempB))
    
    if tempA !=None:
        for i in tempA:
            if i in new_vertex_followlist:
                if b in new_vertex_followlist[i]:
                    count1+=1
    
    count2 = len(set(tempA).intersection(tempB))
    count3 = len(set(new_sink_followlist[a]).intersection(new_sink_followlist[b]))
    
    if tempB != None:
        for i in tempB:
            if i in new_vertex_followlist:
                if a in new_vertex_followlist[i]:
                    count4+=1
      
    totalA = set().union(tempsinkA,tempA)
    totalB = set().union(tempsinkB,tempB)
    commonFriends = list(set(totalA).intersection(totalB))
    count7=len(commonFriends)
    total_AB = len(totalA.union(totalB))
    if total_AB!=0:
        count5 = len(commonFriends)/total_AB
    
    for commonFriend in commonFriends:
        freq_comfri = 0
        if commonFriend in new_vertex_followlist:
            freq_comfri += new_vertex_dict[commonFriend]
        if commonFriend in new_sink_followlist:
            freq_comfri += new_sink_dict[commonFriend]
        if freq_comfri!=0:
            count6+=1/math.log(freq_comfri+1)
    
    if (a,b) in new_vertex_followquicklist:
        count8 =3
        return (count5,count6,count7,count8)
    if (b,a) in new_vertex_followquicklist:
        count8 =2
        return (count5,count6,count7,count8)
    if count7!=0:
        count8 =1
        return (count5,count6,count7,count8)

#     return (count1,count2,count3,count4,count5)
    return (count5,count6,count7,count8)
findneighbor(4066935,349769)

(0.026981450252951095, 2.4021665662725993, 16, 3)

In [124]:
c,show =0,0
for i in new_vertex_dict.keys():
    show+=1
    if i in new_sink_dict:
        c+=1

print (len(new_vertex_followlist))
print (len(new_vertex_dict))
# 19136/19138

19138
19138


In [138]:
# label 1 part 1
input_train = []
count=0
for i in true_pairs:
    a=countFollowingFollower(i[0],i[1])
    count+=1
    input_train.append(list(a))
    if count%1000000==0:
        print (count)

print (len(input_train))

1000000
2000000
3000000
4000000
5000000
5168055


In [126]:
# label 1 part 2
input_train2 = []
count=0
for i in true_pairs:
    b=findneighbor(i[0],i[1])
    count+=1
    input_train2.append(list(b))
    if count%50000==0:
        print (count)
    
print (len(input_train2))

50000
100000
150000
200000
250000
300000
350000
400000
450000
500000
550000
600000
650000
700000
750000
800000
850000
900000
950000
1000000
1050000
1100000
1150000
1200000
1250000
1300000
1350000
1400000
1450000
1500000
1550000
1600000
1650000
1700000
1750000
1800000
1850000
1900000
1950000
2000000
2050000
2100000
2150000
2200000
2250000
2300000
2350000
2400000
2450000
2500000
2550000
2600000
2650000
2700000
2750000
2800000
2850000
2900000
2950000
3000000
3050000
3100000
3150000
3200000
3250000
3300000
3350000
3400000
3450000
3500000
3550000
3600000
3650000
3700000
3750000
3800000
3850000
3900000
3950000
4000000
4050000
4100000
4150000
4200000
4250000
4300000
4350000
4400000
4450000
4500000
4550000
4600000
4650000
4700000
4750000
4800000
4850000
4900000
4950000
5000000
5050000
5100000
5150000
5168055


In [139]:
# label 1 part1+part2
temptest = np.hstack((input_train,input_train2))
print (len(temptest[0]))

10


In [145]:
# features + label 1
true_label = np.ones(len(temptest))
true_featurewithlabel = np.insert(temptest, 10, values=true_label, axis=1)
print (len(true_featurewithlabel))
print (np.shape(true_featurewithlabel))

5168055
(5168055, 11)


In [129]:
# create false pairs
total_list = list(total_set)
sum = len(total_list)-1
false_pairs = []
for i in range(5168056):
    random1 = randint(0,sum)
    random2 = randint(0,sum)
    if random1 in new_vertex_followlist and random2 in new_vertex_followlist[random1]:
        continue
    false_pairs.append([total_list[random1],total_list[random2]])
print (len(false_pairs))

5168055


In [141]:
# label 0 part 1
input_train_1 = []
count=0
for i in false_pairs:
    a=countFollowingFollower(i[0],i[1])
    count+=1
    input_train_1.append(list(a))
    if count%500000==0:
        print (count)
print (len(input_train_1))

500000
1000000
1500000
2000000
2500000
3000000
3500000
4000000
4500000
5000000
5168055


In [131]:
# label 0 part 2
input_train_2 = []
count=0
for i in false_pairs:
    b=findneighbor(i[0],i[1])
    count+=1
    input_train_2.append(list(b))
    if count%50000==0:
        print (count)

50000
100000
150000
200000
250000
300000
350000
400000
450000
500000
550000
600000
650000
700000
750000
800000
850000
900000
950000
1000000
1050000
1100000
1150000
1200000
1250000
1300000
1350000
1400000
1450000
1500000
1550000
1600000
1650000
1700000
1750000
1800000
1850000
1900000
1950000
2000000
2050000
2100000
2150000
2200000
2250000
2300000
2350000
2400000
2450000
2500000
2550000
2600000
2650000
2700000
2750000
2800000
2850000
2900000
2950000
3000000
3050000
3100000
3150000
3200000
3250000
3300000
3350000
3400000
3450000
3500000
3550000
3600000
3650000
3700000
3750000
3800000
3850000
3900000
3950000
4000000
4050000
4100000
4150000
4200000
4250000
4300000
4350000
4400000
4450000
4500000
4550000
4600000
4650000
4700000
4750000
4800000
4850000
4900000
4950000
5000000
5050000
5100000
5150000


In [142]:
# label 0 part1+part2
false_temptest = np.hstack((input_train_1,input_train_2))
print(len(false_temptest[0]))

10


In [149]:
# label 0 + feature0
false_label = np.zeros(len(false_temptest))
false_featurewithlabel = np.insert(false_temptest, 10, values=false_label, axis=1)
print (len(false_featurewithlabel))
# label 1  +  label 0
together = np.vstack((true_featurewithlabel,false_featurewithlabel))
print(np.shape(together))

5168055
(10336110, 11)


In [150]:
np.savez_compressed("new_result_1000", result = together);

In [151]:
# convert test pairs  to  features
test_output = []
for i in true_testpairs:
    temp =[i[0],i[1]]
    test_output.append(temp)
print(np.shape(test_output))

test_features = []
for i in test_output:
    a=countFollowingFollower(i[0],i[1])
    b=findneighbor(i[0],i[1])
    test_features.append(list(a)+list(b))
print (np.shape(test_features))

(2000, 2)
(2000, 10)


In [152]:
np.savez_compressed("new_test_features", testresult = test_features);